In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit 
from math import pi

In [2]:
#system properties
M=6                           #Amount of fcc 'cells' in 1 dimension of the domain. Determines amount of particles.
density=0.45
Tdes=1.55                     #Desired temperature, set this to the temperature at which measurements are to be taken.

#derived properties
n=4*M*M*M                     #Amount of particles. For increasing M: 4, 32, 108, 256, 500, 864, 1372, 2048, 2916, 4000
volume=n/density
Ldomain=volume**(1/3)
Lcell=Ldomain/M

#simulation parameters
h=0.004                       #Timestep size in reduced units.
teq=10                        #Equilibration time in reduced units.
tmeas=2                       #Time for one measurement in reduced units.
NM= 50                        #Number of measurements to be taken, total simulated time is equilibrium time+NM*measurement time.
r_cut_off = 3                 #Cut of distance in reduced units. Only particles within this distance are taken into account for
r_cut_off2 = r_cut_off**2     #the force.

In [3]:
#initiation of position
pos=np.zeros((3,n),dtype='float')     #position of n particles as a vector of length 3 results in a 3xn matrix

#By looping over each cell in each direction, 4 particles can by assigned in each cell, located at the corner and the middle of 
#three adjacent faces.
q=0
for i in range(M):
    for j in range(M):
        for k in range(M):
            pos[:,q]=[Lcell*i,Lcell*j,Lcell*k]
            q+=1
            pos[:,q]=[Lcell/2+i*Lcell,j*Lcell,Lcell/2+k*Lcell]
            q+=1
            pos[:,q]=[Lcell/2+i*Lcell,Lcell/2+j*Lcell,k*Lcell]
            q+=1
            pos[:,q]=[i*Lcell,Lcell/2+j*Lcell,Lcell/2+k*Lcell]
            q+=1


In [4]:
#initiation of momentum
mom=np.zeros((3,n),dtype='float')     #momentum of n particles as a vector of length 3 results in a 3xn matrix
mu, sigma = 0, 1                      #mean and standard deviation, these values are arbitrary (sigma should not be 0 though)

#For each component of the momentum, n values are taken from a normal distribution. Then, the mean is taken and subtracted from
#the momentum of each particle. This guarantees a 0 nett velocity of the particle 'cloud'.
for i in range(3):
    mom[i,:]=np.random.normal(mu, sigma,n)
    mean=np.mean(mom[i,:])
    mom[i,:]-=mean

#The actual temperature of the system is calculated, after which the momentum is rescaled such that the actual temperature
#equals the desired temperature.
T=1/3*np.mean(mom[0,:]**2 + mom[1,:]**2 + mom[2,:]**2)
mom=mom*(Tdes/T)**0.5

In [5]:
#definition force and virial calculation scheme
@jit  
def force(pos,n,Ldomain):             
    F = np.zeros((3,n),dtype='float') #Force on each of n particles as a vector of length 3 results in a 3xn matrix.
    virial = 0                        #Initiation of the virial term which is required for the virial equation (pressure).
    for i in range(n):
        for j in range(i):            #By using Newton's 3rd law, we can evaluate only (roughly) half of the matrix.
            dU = 0
            dx = (pos[0,i]-pos[0,j])
            dy = (pos[1,i]-pos[1,j])
            dz = (pos[2,i]-pos[2,j])
            
            dx = dx - np.rint(dx/Ldomain)*Ldomain     #implementation of the periodic boundary condition by using the
            dy = dy - np.rint(dy/Ldomain)*Ldomain     #'nearest neighbour' principle: If the particle is more than half the
            dz = dz - np.rint(dz/Ldomain)*Ldomain     #domain length away, a particle from a neighbouring domain is closer
                                                      #and is taken into account.
            dr2 = dx*dx + dy*dy + dz*dz

            if dr2 <= r_cut_off2:                     #Only particles within a sphere of radius r are taken into account.
                ir2 = 1/dr2
                ir6 = ir2 * ir2 * ir2
                ir12 = ir6 * ir6
                
                dU = 24 * ir2 * (2 * ir12 - ir6)      #-dU/dr /r term needed for the force calculations.
                
                F[0,i]+=dU*dx
                F[1,i]+=dU*dy
                F[2,i]+=dU*dz
                
                F[0,j]-=dU*dx                         #Newton's 3rd law makes life easier!
                F[1,j]-=dU*dy
                F[2,j]-=dU*dz
                
            virial-= dr2 * dU                         #The virial contribution is calculated as: -(-dU/dr /r)*r^2=r*dU/dr for
                                                      #the current particle and added to the total.
    return F, virial

In [6]:
#initiation forces and virial
F, virial= force(pos, n, Ldomain)

In [7]:
#equilibration of the system
for i in range(np.int(teq/h)):
    #Determines the actual temperature of the system from the average momentum, and then multiplies the momentum
    #by a correcting factor to force the temperature at the desired temperature.
    T = 1/3*np.mean(mom[0,:]*mom[0,:] + mom[1,:]*mom[1,:] + mom[2,:]*mom[2,:])
    mom = (Tdes/T)**0.5*mom
    
    #Time integration by using the Verlet/leapfrog algorithm.
    mom += 0.5*F*h  
    pos += mom*h
    pos = np.mod(pos,Ldomain)
    F, virial = force(pos,n, Ldomain)
    mom += 0.5*F*h


In [8]:
#'release' of the system and subsequent measurements

#Initiation of measurable quantities. Vectors of length NM to contain NM measurements.
virialT = np.zeros(NM, dtype='float')         #'Time integrated' virial term
K = np.zeros(np.int(tmeas/h), dtype='float')  #Kinetic energy of the system for current measurement loop as a function of time.
KT = np.zeros(NM, dtype='float')              #Time averaged Kinetic energy of the system in reduced units.
dKT = np.zeros(NM, dtype='float')             #Time averaged Kinetic energy fluctiation of the system in reduced units.
Tmeas = np.zeros(NM, dtype='float')           #Temperature as measured at the end of the measurement loop in reduced units.
D = np.zeros(NM, dtype='float')               #Difusion coefficient of the system in reduced units.

for j in range(NM):                           #The measurement loop is executed for each measurement.
    disp = np.zeros((3,n), dtype='float')     #We keep track of the nett displacement of each particle in each direction (3xn)
    for i in range(np.int(tmeas/h)):
        #Time integration by using the Verlet/leapfrog algorithm.
        mom += 0.5*F*h  
        pos += mom*h
        pos = np.mod(pos,Ldomain)
        disp += mom*h                         #Note we can't just take the difference in pos due to the periodic BC.
        F, virial   = force(pos,n, Ldomain)
        mom += 0.5*F*h 
        
        #Taking of measurements for the entire duration of the simulation:
        virialT[j]+=virial
        K[i]=0.5*np.sum(mom[0,:]*mom[0,:] + mom[1,:]*mom[1,:] + mom[2,:]*mom[2,:])
   
    #Measuring the appropriate properties at the end of the current simulation loop.
    KT[j]=np.mean(K)
    dKT[j]=np.mean((K-KT[j])*(K-KT[j]))
    Tmeas[j]=2/(3*n)*np.mean(K)
    D[j] = np.mean(sum(disp*disp))            #From <r^2>=Dt, should still be divided by t, but that can be done outside the loop.
    
    
virialT = virialT/(tmeas/h)                   #Transforming the 'time integrated' virial term into time averaged by dividing by
                                              #the number of time steps.
D = D/tmeas                                   #Completes the calculation of D.          

In [9]:

cut_off_term = 48/9*(r_cut_off)**(-9) - 8*(r_cut_off)**(-3)           #calculation of the last term in the virial equation.
P = (1 - virialT/(3*Tmeas*n)) + 2*pi*n/(3*Tmeas*volume)*cut_off_term  #Measure of the pressure for each measurement using virial eq.
Pav = np.mean(P)                              #Average over all measurements, gives a good indication of the pressure of the system.
Perror = np.std(P)                            #The deviation from the average gives a good measure of the spread and thus error
                                              #in the measurement of the pressure.
Cv = -3/(3*n*dKT/(KT*KT)-2)                   #Specific heat per particle from Lebowitz' equation. 
Cvav = np.mean(Cv)                            #Average over all measurements.
Cverror = np.std(Cv)                          #Standard deviation provides the error in the measurement.

Dav = np.mean(D)
Derror = np.std(D)

Tav = np.mean(Tmeas)
Tvar = np.sqrt(n)*np.std(Tmeas)               #We measure the variation instead of the standard deviation to give an indication of
                                              #the spread of the temperature (should not decrease by increasing amount of measurements)
print(Tav,Tvar,Pav,Perror,Cvav,Cverror,Dav,Derror)

1.53700842583 0.165285965502 0.546130478711 0.0649927110652 1.79965563733 0.0943916747065 1.89031216091 0.159189815991
